## 循环神经网络(RNN)
循环神经网络主要用于解决分类问题，虽然提取图片特征没有卷积神经网络强大,但是也可以用于解决分类问题

### 网络模型

In [1]:
from torch import nn
class Rnn(nn.Module):
    def __init__(self):
        super(Rnn,self).__init__()
        self.rnn = nn.RNN(
            # 指定输入数据的特征数
            input_size = 28,

            # 指定最后隐藏层的输出特征数
            hidden_size = 128,

            # 指定循环层堆叠的数量，默认为1
            num_layers = 1,

            # 默认输出维度(seq,batch,feature) seq序列长度 batch数据批次数量 feature数据输入输出特证数、
            # 为True时对应输出(batch,seq,feature)
            batch_first = True
        )
        self.output = nn.Linear(128,10)

    def forward(self,x):
        # H0 默认初始值为0 所以传入None
        output,_ = self.rnn(x,None)

        # 分类问题，所以提取最后一个序列结果作为输出
        output = self.output(output[:,-1,:])

        return output



### 加载数据

In [2]:
import torchvision
train_data = torchvision.datasets.MNIST(
    root = "../../../datas/",
    train = True,
    download = True,
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean = [0.5],std = [0.5]),
    ])

) 

In [3]:
test_data = torchvision.datasets.MNIST(
    root = "../../../datas/",
    train = False,
    download = True,
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean = [0.5],std = [0.5]),
    ])
) 

In [4]:
train_data.__len__()

60000

In [5]:
test_data.__len__()

10000

In [6]:
img , target = train_data[0]
img.shape

torch.Size([1, 28, 28])

### 损失函数与优化器

In [7]:
from pyexpat import model
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Rnn()
model.to(device)

loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)

optimizer = torch.optim.Adagrad(model.parameters(),lr=0.1)

### 超参数

In [8]:
epoch = 30
test_data_size = len(test_data)

### 训练

In [9]:
from torch.utils.data import DataLoader
train_load = DataLoader(train_data,batch_size=64,shuffle=True,num_workers=4,pin_memory=True)
test_load = DataLoader(test_data,batch_size=64,num_workers=4,pin_memory=True)

for i in range(epoch):
    print("-------------第{}轮训练开始------------".format(i+1))
    # 训练步骤开始
    model.train()
    for data in train_load:
        imgs,targets = data
        imgs = imgs.view((-1,28,28))
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = model(imgs)
        loss = loss_fn(outputs,targets)
        
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print("训练轮数：{},loss:{}".format(i+1,loss.item()))
        
    # 测试步骤
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_load:
            imgs,targets = data
            imgs = imgs.view((-1,28,28))
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy+=accuracy
    print("整体测试集上的loss:{}".format(total_test_loss))
    print("整体测试集上的正确率:{}".format(total_accuracy/test_data_size))

    # 保存模型
    if i == epoch-1:
        torch.save(model.state_dict(),"./models/minist_rnn_model.pth")
        print("模型已保存")


-------------第1轮训练开始------------
训练轮数：1,loss:2.337773561477661
整体测试集上的loss:367.30159282684326
整体测试集上的正确率:0.0957999974489212
-------------第2轮训练开始------------
训练轮数：2,loss:2.338036298751831
整体测试集上的loss:367.112202167511
整体测试集上的正确率:0.0982000008225441
-------------第3轮训练开始------------
训练轮数：3,loss:2.3104395866394043
整体测试集上的loss:365.6539719104767
整体测试集上的正确率:0.12029999494552612
-------------第4轮训练开始------------
训练轮数：4,loss:2.333613634109497
整体测试集上的loss:364.7496325969696
整体测试集上的正确率:0.0982000008225441
-------------第5轮训练开始------------
训练轮数：5,loss:2.3570504188537598
整体测试集上的loss:365.20030903816223
整体测试集上的正确率:0.11349999904632568
-------------第6轮训练开始------------
训练轮数：6,loss:2.310353994369507
整体测试集上的loss:363.5524830818176
整体测试集上的正确率:0.09589999914169312
-------------第7轮训练开始------------
训练轮数：7,loss:1.5788441896438599
整体测试集上的loss:223.84800696372986
整体测试集上的正确率:0.4569000005722046
-------------第8轮训练开始------------
训练轮数：8,loss:1.443682312965393
整体测试集上的loss:225.0246695280075
整体测试集上的正确率:0.4161999821662903
--------

### 预测

In [10]:
# 模型加载
model = Rnn()
model_status = torch.load("./models/minist_rnn_model.pth")
model.load_state_dict(model_status)

<All keys matched successfully>

In [42]:
img,target = test_data[0]

In [43]:
target

7

In [44]:
model(img).argmax()

tensor(7)